In [1]:

# Import common dependencies
import pandas as pd  # noqa
import numpy as np
import matplotlib  # noqa
import matplotlib.pyplot as plt
import datetime  # noqa
import PIL  # noqa
import glob  # noqa
import pickle  # noqa
from pathlib import Path  # noqa
from scipy import misc  # noqa
import sys
import tensorflow as tf
import pdb
TRADE_COST_FRAC = .003


# Import common dependencies
import torch
import pandas as pd  # noqa
import numpy as np
import matplotlib  # noqa
import matplotlib.pyplot as plt
import datetime  # noqa
import PIL  # noqa
import glob  # noqa
import pickle  # noqa
from pathlib import Path  # noqa
from torch import nn
from torch.nn import init
from torch.autograd import Variable  # noqa
from scipy import misc  # noqa
from zoneout import ZoneoutLSTMCell

USE_CUDA = 1


def cuda(obj):
    if USE_CUDA:
        if isinstance(obj, tuple):
            return tuple(cuda(o) for o in obj)
        elif isinstance(obj, list):
            return list(cuda(o) for o in obj)
        elif hasattr(obj, 'cuda'):
            return obj.cuda()
    return obj


def tovar(*arrs, **kwargs):
    tensors = [(torch.from_numpy(a) if isinstance(a, np.ndarray) else a) for a in arrs]
    vars_ = [torch.autograd.Variable(t, **kwargs) for t in tensors]
    if USE_CUDA:
        vars_ = [v.cuda() for v in vars_]
    return vars_[0] if len(vars_) == 1 else vars_


def tonumpy(*vars_):
    arrs = [(v.data.cpu().numpy() if isinstance(v, torch.autograd.Variable) else
             v.cpu().numpy() if torch.is_tensor(v) else v) for v in vars_]
    return arrs[0] if len(arrs) == 1 else arrs

EPSILON = 1e-10
ADV_MULT = 1e-3

/home/lee/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


ModuleNotFoundError: No module named 'zoneout'

In [3]:
uni_tokens = set()
uni_commands = set()
uni_actions = set()
fname = 'tasks_with_length_tags.txt'
with open(fname) as f:
    content = f.readlines()
content2 = [c.split(' ') for c in content]
# you may also want to remove whitespace characters like `\n` at the end of each line
commands = []
actions = []
content = [l.replace('\n', '') for l in content]
commands = [x.split(':::')[1].split(' ')[1:-1] for x in content]
actions = [x.split(':::')[2].split(' ')[1:-2] for x in content]
structures = [x.split(':::')[3].split(' ')[2:] for x in content]

structures = [[int(l) for l in program] for program in structures]
#actions = [[wd.replace('\n', '') for wd in res] for res in actions]

In [6]:
structures

[[1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [4],
 [4],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [2],
 [2],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [8, 8],
 [2, 2],
 [2, 2],
 [4, 4],
 [2, 2],
 [3, 3],
 [8, 8],
 [2, 2],
 [3, 3],
 [8, 8],
 [1, 1],
 [1, 1],
 [2, 2],
 [3, 3],
 [8, 8],
 [2, 2],
 [3, 3],
 [2, 2],
 [2, 2],
 [1, 1],
 [3, 3],
 [3, 3],
 [2, 2],
 [4, 4],
 [1, 1],
 [1, 1],
 [8, 8],
 [8, 8],
 [3, 3],
 [1, 1],
 [8, 8],
 [2, 2],
 [8, 8],
 [3, 3],
 [8, 8, 8],
 [2, 2, 2],
 [2, 2, 2],
 [4, 4, 4],
 [2, 2, 2],
 [3, 3, 3],
 [8, 8, 8],
 [2, 2, 2],
 [3, 3, 3],
 [8, 8, 8],
 [1, 1, 1],
 [1, 1, 1],
 [2, 2, 2],
 [3, 3, 3],
 [8, 8, 8],
 [2, 2, 2],
 [3, 3, 3],
 [2, 2, 2],
 [2, 2, 2],
 [1, 1, 1],
 [3, 3, 3],
 [3, 3, 3],
 [2, 2, 2],
 [4, 4, 4],
 [1, 1, 1],
 [1, 1, 1],
 [8, 8, 8],
 [8, 8, 8],
 [3, 3, 3],
 [1, 1, 1],
 [8, 8, 8],
 [2, 2, 2],
 [8, 8, 8],
 [3, 3, 3],
 [2, 2, 2, 2],
 [2, 2, 3, 3, 3],
 [2, 2, 1, 1],
 [2, 2, 1, 1, 1],
 [2, 2, 2, 2],
 

In [ ]:
def build_fmap_invmap(unique, num_unique):
    fmap = dict(zip(range(num_unique), unique))
    invmap = dict(zip(unique, range(num_unique)))
    return fmap, invmap

In [ ]:
for li in content2:
    for wd in li:
        uni_tokens.add(wd)

In [ ]:
for li in commands:
    for wd in li:
        uni_commands.add(wd)

In [ ]:
for li in actions:
    for wd in li:
        uni_actions.add(wd)

In [ ]:
uni_commands.add('end_command')
uni_actions.add('end_subprogram')
uni_actions.add('end_action')

In [ ]:
num_cmd = len(uni_commands)
num_act = len(uni_actions)
size_emb = 2

In [ ]:
command_map, command_invmap = build_fmap_invmap(uni_commands, num_cmd)
action_map, action_invmap = build_fmap_invmap(uni_actions, num_act)

In [ ]:

# def __init__(self, input_size, hidden_size, bias=True, p=0.5):
#     nn.Module.__init__(self)
#     self.p = p
#     self.W_gates = nn.Parameter(
#             T.randn(input_size + hidden_size, 4 * hidden_size) * 0.1)
#     self._hidden_size = hidden_size
#     if bias:
#         self.b_gates = nn.Parameter(T.zeros(4 * hidden_size))

In [ ]:
hidden_filters = 8
# command_emb = cuda(nn.Embedding(num_cmd+1, size_emb, padding_idx = 0, scale_grad_by_freq=True))
# action_emb = cuda(nn.Embedding(num_act+1, size_emb, padding_idx = 0, scale_grad_by_freq=True))

# first_cell = [ZoneoutLSTMCell(
#                 input_size = size_emb, hidden_size = hidden_filters, bias = True, p = 0.5,
#                 ) for _ in range(2)]
# hidden_cells = [[ZoneoutLSTMCell(
#                 input_size = hidden_filters * 2, hidden_size = hidden_filters, bias = True, p = 0.5,
#                 ) for _ in range(2)] for _ in range(num_layers - 1)]
# cells_encoder = first_cell + hidden_cells
# c1, c2 = zip(*cells_encoder)
# cells_encoder = [c1, c2]
class encoder(nn.Module):
    def __init__(self,num_layers):
        nn.Module.__init__(self)
        self.num_layers = num_layers
        command_emb = cuda(nn.Embedding(num_cmd+1, size_emb, padding_idx = 0, scale_grad_by_freq=True))
        action_emb = cuda(nn.Embedding(num_act+1, size_emb, padding_idx = 0, scale_grad_by_freq=True))

        first_cell_f = [ZoneoutLSTMCell(
                        input_size = size_emb, hidden_size = hidden_filters, bias = True, p = 0.5,
                        )]
        hidden_cells_f = [ZoneoutLSTMCell(
                input_size = hidden_filters * 2, hidden_size = hidden_filters, bias = True, p = 0.5,
                ) for _ in range(num_layers - 1)]
        first_cell_b = [ZoneoutLSTMCell(
                        input_size = size_emb, hidden_size = hidden_filters, bias = True, p = 0.5,
                        )]
        hidden_cells_b = [ZoneoutLSTMCell(
                input_size = hidden_filters * 2, hidden_size = hidden_filters, bias = True, p = 0.5,
                ) for _ in range(num_layers - 1)]
        self.fcells = nn.ModuleList(first_cell_f + hidden_cells_f)
        self.bcells = nn.ModuleList(first_cell_b + hidden_cells_b)
        self.relu = nn.LeakyReLU()
    def forward(self,x, lengths):
        x = tovar(x)
        embedding_forward = command_emb(x)
        maxlen = max(lengths)
        num_layers = self.num_layers
        layer_input = embedding_forward
        for lidx in range(num_layers):

            for 
            layer_input_reversed = [
                torch.cat((layer_input[idx:,lengths[idx]::-1,:], 
                           torch.zeros(maxlen - lengths[idx], size_emb)), 1) 
                for idx in range(layer_input.size()[0])]
            forward_outputs = self.fcells[lidx](layer_input)
            backward_outputs = self.bcells[lidx](layer_input_reversed)
            layer_outputs = torch.cat((forward_outputs, backward_outputs), -1)
            layer_input = self.relu(layer_output)
        return layer_input

In [ ]:
enc = encoder(3)

In [ ]:
enc(np.random.randint(3, size = (4,5)), np.array([2, 3, 1]))

In [ ]:
def generate_command(sub_cmd, num_repeat):
    return sub_cmd * num_repeat

In [ ]:
def process_command(cmd):
    

In [ ]:
uni_commands

In [ ]:
uni_actions

In [ ]:
uni_tokens

In [ ]:
df.shape